In [9]:
class Clause(object):
    """A definite clause"""

    def __init__(self, head, body=[]):
        """clause with atom head and lost of atoms body"""
        self.head = head
        self.body = body

    def __str__(self):
        """returns the string representation of a clause.
        """
        if self.body:
            return self.head + " <- " + " & ".join(self.body) + "."
        else:
            return self.head + "."


In [10]:
class Askable(object):
    """An askable atom"""

    def __init__(self, atom):
        """clause with atom head and lost of atoms body"""
        self.atom = atom

    def __str__(self):
        """returns the string representation of a clause."""
        return "askable " + self.atom + "."


In [11]:
def yes(ans):
    """returns true if the answer is yes in some form"""
    return ans.lower() in ['yes', 'yes.', 'oui', 'oui.', 'y', 'y.']    # bilingual


In [12]:
from lib.display import Displayable


class KB(Displayable):
    """A knowledge base consists of a set of clauses.
    This also creates a dictionary to give fast access to the clauses with an atom in head.
    """
    def __init__(self, statements=[]):
        self.statements = statements
        self.clauses = [c for c in statements if isinstance(c, Clause)]
        self.askables = [c.atom for c in statements if isinstance(c, Askable)]
        self.atom_to_clauses = {}  # dictionary giving clauses with atom as head
        for c in self.clauses:
            if c.head in self.atom_to_clauses:
                self.atom_to_clauses[c.head].add(c)
            else:
                self.atom_to_clauses[c.head] = {c}

    def clauses_for_atom(self, a):
        """returns set of clauses with atom a as the head"""
        if a in self.atom_to_clauses:
            return self.atom_to_clauses[a]
        else:
            return set()

    def __str__(self):
        """returns a string representation of this knowledge base.
        """
        return '\n'.join([str(c) for c in self.statements])


In [13]:
triv_KB = KB([
    Clause('i_am', ['i_think']),
    Clause('i_think'),
    Clause('i_smell', ['i_exist'])
])

elect = KB([
    Clause('light_l1'),
    Clause('light_l2'),
    Clause('ok_l1'),
    Clause('ok_l2'),
    Clause('ok_cb1'),
    Clause('ok_cb2'),
    Clause('live_outside'),
    Clause('live_l1', ['live_w0']),
    Clause('live_w0', ['up_s2', 'live_w1']),
    Clause('live_w0', ['down_s2', 'live_w2']),
    Clause('live_w1', ['up_s1', 'live_w3']),
    Clause('live_w2', ['down_s1','live_w3' ]),
    Clause('live_l2', ['live_w4']),
    Clause('live_w4', ['up_s3','live_w3' ]),
    Clause('live_p_1', ['live_w3']),
    Clause('live_w3', ['live_w5', 'ok_cb1']),
    Clause('live_p_2', ['live_w6']),
    Clause('live_w6', ['live_w5', 'ok_cb2']),
    Clause('live_w5', ['live_outside']),
    Clause('lit_l1', ['light_l1', 'live_l1', 'ok_l1']),
    Clause('lit_l2', ['light_l2', 'live_l2', 'ok_l2']),
    Askable('up_s1'),
    Askable('down_s1'),
    Askable('up_s2'),
    Askable('down_s2'),
    Askable('up_s3'),
    Askable('down_s2')
])

# print(kb)


In [14]:
def ask_askables(kb):
    return {at for at in kb.askables if yes(input("Is " + at + " true? "))}


def fixed_point(kb):
    """Returns the fixed point of knowledge base kb.
    """
    fp = ask_askables(kb)
    added = True
    while added:
        added = False  # added is true when an atom was added to fp this iteration
        for c in kb.clauses:
            if c.head not in fp and all(b in fp for b in c.body):
                fp.add(c.head)
                added = True
                kb.display(2, c.head, "added to fp due to clause", c)
    return fp


In [17]:
def test(kb=triv_KB, fixedpt = {'i_am', 'i_think'}):
    fp = fixed_point(kb)
    assert fp == fixedpt, "kb gave result " + str(fp)
    print("Passed unit test")

    
if __name__ == "__main__":
    test()


i_think added to fp due to clause i_think.
i_am added to fp due to clause i_am <- i_think.
Passed unit test


In [16]:
elect.max_display_level=3  # give detailed trace

fixed_point(elect)


Is up_s1 true? y
Is down_s1 true? n
Is up_s2 true? y
Is down_s2 true? n
Is up_s3 true? y
Is down_s2 true? n
light_l1 added to fp due to clause light_l1.
light_l2 added to fp due to clause light_l2.
ok_l1 added to fp due to clause ok_l1.
ok_l2 added to fp due to clause ok_l2.
ok_cb1 added to fp due to clause ok_cb1.
ok_cb2 added to fp due to clause ok_cb2.
live_outside added to fp due to clause live_outside.
live_w5 added to fp due to clause live_w5 <- live_outside.
live_w3 added to fp due to clause live_w3 <- live_w5 & ok_cb1.
live_w6 added to fp due to clause live_w6 <- live_w5 & ok_cb2.
live_w1 added to fp due to clause live_w1 <- up_s1 & live_w3.
live_w4 added to fp due to clause live_w4 <- up_s3 & live_w3.
live_p_1 added to fp due to clause live_p_1 <- live_w3.
live_p_2 added to fp due to clause live_p_2 <- live_w6.
live_w0 added to fp due to clause live_w0 <- up_s2 & live_w1.
live_l2 added to fp due to clause live_l2 <- live_w4.
lit_l2 added to fp due to clause lit_l2 <- light_l2 

{'light_l1',
 'light_l2',
 'lit_l1',
 'lit_l2',
 'live_l1',
 'live_l2',
 'live_outside',
 'live_p_1',
 'live_p_2',
 'live_w0',
 'live_w1',
 'live_w3',
 'live_w4',
 'live_w5',
 'live_w6',
 'ok_cb1',
 'ok_cb2',
 'ok_l1',
 'ok_l2',
 'up_s1',
 'up_s2',
 'up_s3'}